# Agentic RAG Playground by LlamaIndex

In [1]:
import datasets
from llama_index.core.schema import Document

In [2]:
# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        text="\n".join([
            f"Name: {guest_dataset['name'][i]}",
            f"Relation: {guest_dataset['relation'][i]}",
            f"Description: {guest_dataset['description'][i]}",
            f"Email: {guest_dataset['email'][i]}"
        ]),
        metadata={"name": guest_dataset['name'][i]}
    )
    for i in range(len(guest_dataset))
]

In [3]:
# Initialize the tool
from src.tools import GuestInfoRetrieverTool
guest_info_tool = GuestInfoRetrieverTool(docs)

In [4]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.ollama import Ollama
from llama_index.core.tools import FunctionTool
from src.tools import GuestInfoRetrieverTool

# Initialize the guest info retriever tool
guest_info_retriever = GuestInfoRetrieverTool(docs)

# Create a proper FunctionTool
guest_info_tool = FunctionTool.from_defaults(
    fn=guest_info_retriever.get_guest_info,
    name="guest_info",
    description="Retrieve information about guests attending the gala by their name"
)

# Initialize the LLM which one is available in the local system
llm = Ollama(model="llama3:latest")

# Create the agent with minimal configuration
alfred = ReActAgent.from_tools(
    [guest_info_tool],
    llm=llm,
    verbose=True,
    max_iterations=0  # to limit how many iterations to get the final result.
)

# Example query
response = await alfred.aquery("Tell me about our guest named 'Lady Ada Lovelace'.")

# Check if the response contains the answer
if "Lady Ada Lovelace" in response.response:
    print("🎩 Alfred's Response:")
    print(response.response)
else:
    print("🎩 Alfred did not find the answer.")

> Running step 25fbc111-0153-43ad-9913-400dbcc0e870. Step input: Tell me about our guest named 'Lady Ada Lovelace'.
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: guest_info
Action Input: {'query': 'Lady Ada Lovelace'}
Observation: Name: Ada Lovelace
Relation: best friend
Description: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine.
Email: ada.lovelace@example.com
> Running step a832f0c1-c608-4795-b247-ea222830a1d4. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Lady Ada Lovelace is indeed a renowned mathematician and friend, celebrated for her pioneering work in mathematics and computing. She is often referred to as the first computer programmer

In [ ]:
🎩 Alfred's Response:
Lady Ada Lovelace is an esteemed mathematician and friend, renowned for her pioneering work in mathematics and computing. She is celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine. Her email is ada.lovelace@example.com.

🎩 Alfred's Response:
Lady Ada Lovelace is indeed a renowned mathematician and friend, celebrated for her pioneering work in mathematics and computing. She is often referred to as the first computer programmer due to her work on Charles Babbage's Analytical Engine.